In [1]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers

## Создание модели и предсказание следующего токена - 5 баллов
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и получить следующий токен через жадную генерацию!

Для загрузки модели и токенайзера вам помогут функции `.from_pretrained`

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [3]:
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name) # Ваш код здесь
tokenizer = AutoTokenizer.from_pretrained(model_name) # ваш код здесь


text = "This is a sample text"

# Нужно преобразовать text с помощью tokenizer() и подать это в model.forward() (он же просто model())
# после этого мы получим logits [batch_size = 1, seq_len, d_model]
# По этому тензору нужно предсказать следующее слово!

inputs = tokenizer(text, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits
next_token_idx: int = torch.argmax(logits[0][-1])

next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"

## Используем Generate - 5 баллов

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [4]:
text = "This is still a sample text, but"
inputs = tokenizer(text, return_tensors="pt")

results: list[str] = []
for i in range(10):
    gens: torch.Tensor = model.generate(**inputs, top_k=20, temperature=0.9, repetition_penalty=1.2, max_new_tokens=10, do_sample=True)
    generation: str = tokenizer.decode(gens[0]) # сгенерированный текст
    results.append(generation)

assert len(set(results)) > 1, "Все генерации получились одинаковыми, проверьте опции генерации и флаг do_sample!"

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## Generate Batched - 5
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда 

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side в конструкторе токенизатора.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name) # ваш код здесь
tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
texts = ["This is a sample text", "I'm really tired and this is just about"]

# Внимание! В данном задании нужна жадная генерация!

# Соберите оба текста в один батч и положите результаты генерации в 
# batched_generations
batched_generations: List[str] = []

batch = tokenizer(texts, return_tensors="pt", padding=True, padding_side="left")

with torch.no_grad():
    gens_tokens = model.generate(**batch)

gens_decoded = tokenizer.batch_decode(gens_tokens, skip_special_tokens=True)
batched_generations += gens_decoded

# Пройдитесь по каждому сэмплу по отдельности и положите результаты генерации 
# в single_generations
single_generations: List[str] = []

for text in texts:
    tokenized = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        gen_tokens = model.generate(**tokenized)

    gen_decoded = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    single_generations.append(gen_decoded)

assert len(batched_generations) == 2 and len(single_generations) == 2
for s, b in zip(batched_generations, single_generations):
    assert s == b

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# Скоринг, Perplexity - 10 баллов

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [7]:
def calc_probs_simple(input_ids: torch.Tensor, log_probs: torch.Tensor) -> torch.Tensor:
    # сделал неэффективное, но верное решение в лоб для подбора параметров для torch.gather
    result = []

    for batch_idx in range(input_ids.shape[0]):
        inp = input_ids[batch_idx][1:]
        lp = log_probs[batch_idx]
        
        batch_log_probs = []
        for idx, item in enumerate(inp):
            batch_log_probs.append(lp[idx][item].item())

        result.append(batch_log_probs)

    return torch.tensor(result)


print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

    # 1. Нужно обрезать logits по длине, т.к. для предсказаний по последнему токену нечего считать
    logits_trunc = logits[:, :-1, :]

    # 2. Превращаем logits в log_probs
    log_probs = torch.log_softmax(logits_trunc, dim=-1)

    # 3. Берем вероятности следующих токенов, т.к. по вектору i-й позиции мы предсказываем токен на позиции (i + 1)
    # для этого нам поможет torch.gather
    expected_log_probs = calc_probs_simple(inputs["input_ids"], log_probs)

    # !вопрос! это такой сложной конструкцией должно оформляться? или есть более элегантное и простое решение?
    index = inputs["input_ids"][:, 1:].unsqueeze(-1)
    next_token_log_probs = torch.gather(input=log_probs, dim=2, index=index).squeeze(-1) # batch_size x seq_len - 1

    assert torch.all(expected_log_probs==next_token_log_probs)

    # 4. Считаем вероятности и perplexity!
    prob = torch.exp(next_token_log_probs.sum())
    ppl = torch.exp((-1 / next_token_log_probs.shape[-1]) * next_token_log_probs.sum())

    # должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl
    expected_prob = torch.tensor(2.1783e-14)
    expected_ppl = torch.tensor(51.0)

    assert torch.isclose(prob, expected_prob)
    assert torch.isclose(ppl, expected_ppl, rtol=1e-3)

Beginning of sentence (BOS) token = `<|endoftext|>`
End of sentence (EOS) token  = `<|endoftext|>`


# Вопросы - 5 баллов

**Ответьте на вопрсоы текстом прямо здесь!**


1. Какое значение P(X) вероятности текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.

    Ответ: 1 - максимальное значение вероятности [0..1]. Чем ближе к 1 тем более модель уверена в ответе

2. Какое значение перплексии текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.

    Ответ: тоже 1, но это минимальное значение Perplexity. Чем выше значение, тем больше "удивлена" модель заданной генерации


# Chat-Models

# Формат - 5 баллов
Как мы обсуждали на лекции, все chat-модели принимают входы в своем особом формате.
Он может быть описан текстом, а может быть заложен в шаблон, который доступен через `tokenizer.apply_chat_template`

In [8]:
from dotenv import load_dotenv

try:
    load_dotenv("../.env")  # load HF_TOKEN for downloading google/gemma-2b-it
except:
    pass

In [9]:
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
model_name = "google/gemma-2b-it" # smaller model to fit 16GB

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.half).to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
def move_to_device(d):
    for k, v in d.items():
        d[k] = v.to(device)
    return d

Давайте посмотрим, как chat модель отработает на обычном тексте. Используйте для генерации сэмплинг и kv cache, выведите 5 результатов генерации.

In [ ]:
text = "hello how are you"
inputs = tokenizer(text, return_tensors="pt")

# !вопрос! как включить kv cache?

for i in range(5):
    generated_tokens = model.generate(**move_to_device(inputs), use_cache=True, do_sample=True)
    generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    print(generated_text)
    print("====" * 3)

hello how are you doing today?

As a large language model, I do not experience time in the same way that
hello how are you doing today?

I am doing well, thank you for asking!  I hope you are having


Видим, что текст зачастую выходит мусорный. Это потому что формат входных данных сильно отличается от того, что модель видела на обучении.
Как мы уже обсуждали, у всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели.

Не забудьте про опцию add_generation_prefix - она добавляет часть формата, после которой ожидается ответ модели!

In [12]:
messages = [
    {"role": "user", "content": "hello"}, 
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]

prefix = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

reference_llama = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

reference_gemma = """<bos><start_of_turn>user
hello<end_of_turn>
<start_of_turn>model
I'm good. How can I help you today<end_of_turn>
<start_of_turn>user
I love you<end_of_turn>
<start_of_turn>model"""

reference = reference_llama if "Llama" in model.name_or_path else reference_gemma
assert prefix.strip() == reference.strip()

Давайте посмотрим, что нам ответит модель!

In [13]:
inputs = tokenizer(prefix, return_tensors="pt")
gen_tokens = model.generate(**inputs)
gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
print(gen_text)

user
hello
model
I'm good. How can I help you today
user
I love you
model
I'm happy to hear that! How can I help you today? Is there anything I can


## Benchmark - 15

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа

In [14]:
from datasets import load_dataset

mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[1]

{'question': 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices?',
 'subject': 'global_facts',
 'choices': ['About $300', 'About $3k', 'About $8k', 'About $15k'],
 'answer': 1}

Наша задача здесь решить задачу многоклассовой классификации.
Для этого нужно посчитать 
$$P(choices_i | question)$$
т.е. для посчитать вероятность каждого варианта ответа для вопроса. Мы это уже делали кодом выше!

После этого давайте брать самый вероятный ответ и считать, что модель его выбрала.
После этого давайте посчитаем accurracy, т.е. долю правильных ответов.
Вместо вероятностей для подсчета лучше использовать logprobs.

Итого, что нужно сделать:
1. Пройтись по датасету, для каждого question и каждого из соответствующих choices получить самый вероятный ответ.
2. Посчитать итоговый accuracy

**Важно**
1. Выше мы уже написали скоринг текста с помощью LLM, для этого задания можно адаптировать функцию.
2. Если делаете варианты с батчеванием помните: длины choices могут быть разными! Нужно не считать вероятности по паддингам. В этом нам помогут attention_masks из выходов `tokenizer()`
3. В данном задании для простоты мы игнорируем формат ответа llama3 и делаем скоринг по f"{question} {answer}"


Попробуйте для начала написать вариант со скорингом для батча размера 1, а потом для батча размера 3 или 5. Код должен корректно работать для батча любого размера и выдавать одинаковую итоговую точность.

За задание, в котором код работает только с батчом размера 1, 2, 4 можно получить **только 10 баллов**

In [15]:
from tqdm.notebook import tqdm
from datasets import Dataset


def sample_to_texts(sample):
    return [sample["question"] + " " + answer for answer in sample["choices"]]

all_samples_formatted = sum([sample_to_texts(sample) for sample in mmlu], [])
#print(*all_samples_formatted[2:6], sep="\n")
# ваш код здесь!

@torch.no_grad()
def calc_texts_log_probs(texts: list[str]) -> list[float]:
    # Задача - найти самый вероятный ответ, то есть сравнить вероятность каждой из опций и выбрать наибольшую
    # Считать вероятности надо только по токенам ответа, т.к. вопрос считается уже заданным и вероятность генерации текста вопроса нам не нужна
    # Но т.к. вероятности по токенам вопроса будут неизменной, можно посчитать по целой строке "вопрос вариант" и выбрать наибольшую
    # Вероятности крайне малые будут, так что опираемся на логарифмы вероятностей log_probs

    inputs = tokenizer(texts, return_tensors="pt", padding=True, padding_side="right")

    logits = model(**inputs).logits
    logits_trunc = logits[:, :-1, :]
    log_probs = torch.log_softmax(logits_trunc, dim=-1)
    index = inputs["input_ids"][:, 1:].unsqueeze(-1)
    next_token_log_probs = torch.gather(input=log_probs, dim=2, index=index).squeeze(-1) # batch_size x seq_len - 1

    # у маски как и у инпутов надо убрать первый элемент, т.к. по первому токену предсказаний нет
    shifted_mask = inputs["attention_mask"][:, 1:]
    next_token_log_probs_masked = next_token_log_probs.masked_fill(~shifted_mask.bool(), 0)

    texts_log_probs = next_token_log_probs_masked.sum(dim=-1)

    return texts_log_probs


class DatasetEvaluator:
    def __init__(self, batch_size: int):
        assert batch_size >= 1
        self.batch_size = batch_size

        self._accuracy = 0.0
        self._num_questions_answered = 0
        self._num_correct_answers = 0
        self._questions = []
        self._batch_texts = []
        self._predicted_answers = []

    def _add_to_batch(self, text: str) -> None:
        self._batch_texts.append(text)

        if len(self._batch_texts) == self.batch_size:
            self._predict_batch()

    def _predict_batch(self):
        if self._batch_texts:
            log_probs = calc_texts_log_probs(self._batch_texts)
            self._predicted_answers += log_probs
            self._process_answers()

            self._batch_texts.clear()

    def _process_answers(self):
        while self._questions:
            question = self._questions[0]
            num_answers = len(question["choices"])
            correct_answer_idx = question["answer"] - 1

            # not all options for a question were calculated
            if len(self._predicted_answers) < num_answers:
                break

            predicted_answers_logprobs = torch.tensor(self._predicted_answers[:num_answers])
            is_prediction_correct = torch.argmax(predicted_answers_logprobs) == correct_answer_idx
            self._accept_answer(is_prediction_correct)

            # remove question and answers from buffer
            self._questions.pop(0)
            self._predicted_answers = self._predicted_answers[num_answers:]

    def _accept_answer(self, correct: bool) -> None:
        self._num_questions_answered += 1

        if correct:
            self._num_correct_answers += 1

        self._accuracy = self._num_correct_answers / self._num_questions_answered

    # Can be called multiple times to accumulate accuracy
    def calculate_dataset_accuracy(self, dataset: Dataset) -> float:
        ds_iterable = tqdm(dataset)

        for entry in ds_iterable:
            # display current accuracy
            ds_iterable.set_description(f"Accuracy: {self._accuracy:.2f}")

            self._questions.append(entry)
            for text in sample_to_texts(entry):
                self._add_to_batch(text)

        self._predict_batch()

        return self._accuracy
    
    def reset(self) -> None:
        self._accuracy = 0.0
        self._num_questions_answered = 0
        self._num_correct_answers = 0

    # for debugging
    def print_results(self) -> None:
        print(f"Batch Size: {self.batch_size} Accuracy: {self._accuracy} Total Questions: {self._num_questions_answered} Correct Answers: {self._num_correct_answers}")


dataset = mmlu.select(range(100))

ds_eval_batch_1 = DatasetEvaluator(batch_size=1)
ds_eval_batch_7 = DatasetEvaluator(batch_size=7)

ds_accuracy_batch_1 = ds_eval_batch_1.calculate_dataset_accuracy(dataset)
ds_accuracy_batch_7 = ds_eval_batch_7.calculate_dataset_accuracy(dataset)

assert ds_accuracy_batch_1 == ds_accuracy_batch_7

print(f"Resulting dataset accuracy: {ds_accuracy_batch_1}") # 0.27 for google/gemma-2b-it, only 0.02 better than random.

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Resulting dataset accuracy: 0.27


In [16]:
# debugging
ds_eval_batch_1.print_results()
ds_eval_batch_7.print_results()

Batch Size: 1 Accuracy: 0.27 Total Questions: 100 Correct Answers: 27
Batch Size: 7 Accuracy: 0.27 Total Questions: 100 Correct Answers: 27


# Вопросы - 5 баллов

**Ответьте на следующие вопросы (5 баллов в сумме)**:
1. Как влияет длина ответа на вероятность ответа при скоринге? Если есть какие-либо проблемы, как бы вы с этим боролись.

Ответ:

Чем длинее ответ, тем меньше будет его вероятность, т.к. даже в идеальной модели вероятность следующего токена будет меньше 1 ввиду энтропии естественного языка. Значит наиболее репрезентативно сравнивать ответы одинаковой или близкой длины

Из проблем можно отметить то, что вероятность целого ответа - очень маленькое число, выходящее за точность float и работать надо с логарифмом

2. Если к началу каждого ответа добавилить метки A) B) C) D) станет ли модель отвечать лучше или хуже?
Стоит ли по-вашему добавлять эти метки?

Ответ: если мы эвалюируем ответы методом подсчета вероятностей как в задании выше, то добавление меток скорее ухудшит качество ответов.

Причины: 
- Чтобы качество было хорошее при наличии меток, тренировочные данные должны содержать этот же тест с вариантами ответа жестко привязанными к меткам, другими словами метка становится частью ответа. Смена порядка ответов при эвалюации будет влиять на качество (префиксы остаются A B C D, а варианты за ними могут быть в любом порядке)
- Добавление префиксов добавляет больше токенов и это уменьшает вероятность каждого из ответов

По вышеуказанным причинам префиксы лучше не добавлять

С другой стороны, если делать эвалюацию более мощных моделей в промпте указывая, что нужно вывести номер\префикс правильного ответа, то предположу, что качество не измениться, а может и улучшиться, за счет того, что генерироваться будет только один токен правильного ответа (уменьшится шанс геренации ответа, которого не было в списке вариантов)